In [1]:
import psycopg2
from psycopg2 import sql

# 替换以下信息为你的数据库信息
database_params = {
    'dbname': 'tpcds',
    'user': 'postgres',
    'password': 'postgres',
    'host': '127.0.0.1',
    'port': '5432'
}

# 尝试连接数据库
try:
    # 使用 **params 将参数传递给 connect 函数
    connection = psycopg2.connect(**database_params)
    
    # 创建一个游标对象
    cursor = connection.cursor()
    
    # 执行一个简单的 SQL 查询
    cursor.execute(sql.SQL("SELECT version()"))
    
    # 获取查询结果
    db_version = cursor.fetchone()
    
    # 打印结果
    print("PostgreSQL 版本:", db_version)

except Exception as e:
    print("连接数据库时发生错误:", e)

finally:
    # 关闭游标和连接
    if cursor:
        cursor.close()
    if connection:
        connection.close()

PostgreSQL 版本: ('PostgreSQL 12.15 (Ubuntu 12.15-1.pgdg18.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit',)


In [2]:
import re

def replace_days_syntax(sql_query):
    # 使用正则表达式匹配带有 'days' 的日期操作，并替换为 'interval' 形式
    modified_sql = re.sub(r"\b(\d+)\s+days\b", r"interval '\1' day", sql_query, flags=re.IGNORECASE)
    return modified_sql

# 示例 SQL 查询
original_query = " \t\t\t\tand (cast('2002-02-12' as date) + 30 days)\n group by i_item_"

# 调用替换函数
modified_query = replace_days_syntax(original_query)

# 输出替换后的查询
print(modified_query)

 				and (cast('2002-02-12' as date) + interval '30' day)
 group by i_item_


In [4]:
# import os
# import glob

# def read_sql_files(folder_path):
#     # 获取指定文件夹下的所有 SQL 文件
#     sql_files = glob.glob(os.path.join(folder_path, '*.sql'))

#     # 读取每个 SQL 文件的内容
#     sql_queries = []
#     for sql_file in sql_files:
#         with open(sql_file, 'r') as file:
#             query = file.read()
#             sql_queries.append(query)

#     return sql_queries

# # 指定包含 SQL 文件的文件夹路径
# folder_path = '/home/ubuntu/data/tpcds-kit/queries/separated'

# # 读取所有 SQL 文件
# queries_ = read_sql_files(folder_path)
# queries = [replace_days_syntax(q) for q in queries_]  #修复识别 days

# # 打印每个查询
# for i, query in enumerate(queries[:2], start=1):
#     print(f"Query {i}:\n{query}\n{'='*30}")

## 重新统计操作算子

In [4]:
import pandas as pd
queries_df=pd.read_csv("./information/queries_filter.csv")
queries_=queries_df['query'].values
queries = [replace_days_syntax(q) for q in queries_]  #修复识别 days
len(queries)

14400

In [5]:
# import psycopg2
# from PGUtils import pgrunner
# import json

# db_params = {
# 'dbname' : 'tpcds',
# 'user' : 'postgres',
# 'password' : 'postgres',
# 'host' : '127.0.0.1',
# 'port' : '5432'
# }

# def extract_operators_from_plan(plan):
#     operators = [step['Node Type'] for step in plan.get('Plans', [])]
#     for nested_plan in plan.get('Plans', []):
#         operators.extend(extract_operators_from_plan(nested_plan))
#     return operators

# def get_execution_plan_operators(query):
#     conn = psycopg2.connect(**db_params)
#     cursor = conn.cursor()

#     try:
#         # 获取执行计划
#         explain_query = f"EXPLAIN (FORMAT JSON) {query}"
#         cursor.execute(explain_query)
#         explain_result = cursor.fetchone()
#         # 解析执行计划中的操作算子类型
#         plan_json = explain_result[0] if isinstance(explain_result, tuple) and len(explain_result) > 0 else explain_result
#         plan_jsons.append(plan_json[0])
#         get_queries.append(query)
#         print("plan_json",plan_json)
#         operators = [plan_json[0]['Plan']['Node Type']]
#         for plan_data in plan_json:
#             operators.extend(extract_operators_from_plan(plan_data['Plan']))

#         return operators

#     finally:
#         cursor.close()
#         conn.close()

# # 统计所有查询的操作算子类型
# all_operators = []
# plan_jsons=[]
# get_queries=[]
# error_sql=[]
# cnt=1
# for query in queries:
#     try:
#         operators = get_execution_plan_operators(query)
#         all_operators.extend(operators)
#         print(cnt, query)
#         cnt += 1
#     except Exception as e:
#         error_sql.append(query)
#         print(f"Error processing query {cnt}: {str(e)}")
#         # 可以选择在这里添加其他处理逻辑，例如记录错误信息或者直接跳过
#         pass
# print("count:",cnt)
# # 统计操作算子类型的数量
# operator_counts = {}
# for operator in all_operators:
#     operator_counts[operator] = operator_counts.get(operator, 0) + 1

# # 打印结果
# print("操作算子类型统计:")
# for operator, count in operator_counts.items():
#     print(f"{operator}: {count} 次")

In [13]:
len(plan_jsons),len(queries)

(14400, 14400)

In [ ]:
import pandas as pd

# 假设 queries 和 plan_json 是两个列表，分别存储查询和对应的执行计划 JSON
# queries = ["SELECT * FROM table1;", "SELECT * FROM table2;"]
# plan_json = [{"plan": "json1"}, {"plan": "json2"}]

# 创建一个 DataFrame
df = pd.DataFrame({"query_id": list(range(1,17814)),"query": get_queries})

# 将 DataFrame 保存为 CSV 文件
df.to_csv("./queries.csv", index=False)
df.head()

,query_id,query
0,1,"select channel, col_name, d_year, d_qoy, i_ca..."
1,2,"select i_item_id, \n avg(ss_quantity) ..."
2,3,"select sum (ss_quantity)\n from store_sales, s..."
3,4,"select i_item_id\n ,i_item_desc \n ..."
4,5,with ss_items as\n (select i_item_id item_id\n...


In [14]:
len(operator_counts),print([k for k,_ in operator_counts.items()])

['Limit', 'Aggregate', 'Gather Merge', 'Sort', 'Nested Loop', 'Index Scan', 'Hash Join', 'Hash', 'Seq Scan', 'Gather', 'WindowAgg', 'Merge Join', 'Materialize', 'Subquery Scan', 'Group', 'Index Only Scan', 'CTE Scan', 'SetOp', 'Append', 'Result', 'Unique', 'Bitmap Heap Scan', 'Bitmap Index Scan']


(23, None)

In [6]:
import json

# 将字典保存为 JSON 文件
with open('./information/operater_counts.json', 'w') as json_file:
    json.dump(operator_counts, json_file)

In [ ]:
with open('./information/operater_counts.json', 'r') as json_file:
    loaded_dict = json.load(json_file)

[k for k,_ in loaded_dict.items()]

['Limit',
 'Aggregate',
 'Gather Merge',
 'Sort',
 'Append',
 'Hash Join',
 'Nested Loop',
 'Hash',
 'Seq Scan',
 'Index Scan',
 'Gather',
 'WindowAgg',
 'Merge Join',
 'Materialize',
 'Subquery Scan',
 'CTE Scan',
 'Group',
 'Index Only Scan',
 'Bitmap Heap Scan',
 'Bitmap Index Scan',
 'SetOp',
 'Result',
 'Unique',
 'Merge Append']

In [6]:
# import psycopg2
# from PGUtils import pgrunner
# import json

# db_params = {
# 'dbname' : 'tpcds',
# 'user' : 'postgres',
# 'password' : 'postgres',
# 'host' : '127.0.0.1',
# 'port' : '5432'
# }

# def extract_operators_from_plan(plan):
#     operators = [step['Node Type'] for step in plan.get('Plans', [])]
#     for nested_plan in plan.get('Plans', []):
#         operators.extend(extract_operators_from_plan(nested_plan))
#     return operators

# def get_execution_plan_operators(query):
#     conn = psycopg2.connect(**db_params)
#     cursor = conn.cursor()

#     try:
#         # 获取执行计划
#         explain_query = f"EXPLAIN (FORMAT JSON) {query}"
#         cursor.execute(explain_query)
#         explain_result = cursor.fetchall()

#         # 解析执行计划中的操作算子类型
#         plan_json = explain_result[0] if isinstance(explain_result, tuple) and len(explain_result) > 0 else explain_result
#         print("plan_json:",plan_json)
#         # 由于可能是多个执行计划，循环处理每个计划
#         operators = [plan_json[0]['Plan']['Node Type']]
#         for plan_data in plan_json:
#             operators.extend(extract_operators_from_plan(plan_data['Plan']))

#         return operators

#     finally:
#         cursor.close()
#         conn.close()

# # 统计所有查询的操作算子类型
# all_operators = []
# cnt=1
# operators = get_execution_plan_operators(replace_days_syntax(error_sql[1]))
# all_operators.extend(operators)
# print(cnt, query)
# # 统计操作算子类型的数量
# operator_counts = {}
# for operator in all_operators:
#     operator_counts[operator] = operator_counts.get(operator, 0) + 1

# # 打印结果
# print("操作算子类型统计:")
# for operator, count in operator_counts.items():
#     print(f"{operator}: {count} 次")